# ABS Monthly Consumer Price Index Indicator 6484

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    find_id,
    get_abs_meta_and_data,
    get_fs_constants,
    get_meta_constants,
    get_plot_constants,
)
from henderson import hma
from plotting import (
    COLOR_AMBER,
    COLOR_BLUE,
    COLOR_RED,
    calc_growth,
    clear_chart_dir,
    finalise_plot,
    line_plot,
    plot_growth,
    set_chart_dir,
)

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
CAT_ID = "6484"
source, CHART_DIR, META_DATA = get_fs_constants(CAT_ID)
abs_dict = get_abs_meta_and_data(CAT_ID, verbose=False)
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col, tdesc_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Retrieving data from the cache file: ABS_CACHE/7a5a63f14a14e9dd35e20efad2c5af41--All-Time-Series-Spreadsheets.zip
Extracting DataFrames from the zip-file ...


## Plot

### Set-up 

In [3]:
def chart_settings(chart_dir: str = CHART_DIR) -> None:
    """Set up charting parameters."""

    set_chart_dir(chart_dir)
    clear_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


chart_settings()
TAREGT_COLOR = "#dddddd"

### Supporting functions

In [4]:
def annualise_rates(series: pd.Series, periods: int = 12) -> pd.Series:
    """Annualise a growth rate for a period.
    Note: returns a percentage (and not a rate)!"""

    return (((1 + series) ** periods) - 1) * 100

### Plot growth - combined line/bar charts

In [5]:
def cpi_growth():
    """Produce CPI growth charts."""

    rows = meta[meta[did_col].str.contains("Index Numbers")]

    for index, row in rows.iterrows():
        # get data for the plot
        did, table = row[did_col], row[table_col]
        series_type, series_id = row[type_col], row[id_col]
        name = did.split(";")[1].strip()
        index = abs_dict[table][series_id]
        growth = calc_growth(index)

        # plot the data
        ax = plot_growth(*growth)
        if ax:
            ax.axhspan(
                2, 3, color=TAREGT_COLOR, zorder=-1, label="Annual inflation target"
            )
            ax.legend(loc="best", fontsize="small")
            long_name = 40
            separator = " " if len(name) < long_name else "\n"
            finalise_plot(
                ax,
                title=f"Price Growth:{separator}{name}",
                ylabel="Per cent",
                rfooter=source,
                lfooter=f"Australia: {series_type.capitalize()} series. ",
                show=SHOW,
            )

In [6]:
cpi_growth()

### Annualised growth

In [7]:
def line_and_target_plot(series: pd.Series, **kwargs) -> None:
    """Finish up plot."""

    ax = series.plot(lw=2, color=COLOR_RED)
    ax.axhspan(2, 3, color=TAREGT_COLOR, zorder=-1, label="Annual inflation target")
    finalise_plot(ax, **kwargs)


def annualised_growth() -> None:
    """Produce annualised growth charts."""

    henderson = 13

    for series_name in (
        "Index Numbers ;  All groups CPI ;  Australia ;",
        "Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;",
    ):
        print(series_name)
        table = "1"
        selector = {series_name: did_col}
        series_id, _ = find_id(meta, selector, verbose=False)
        series_type = (
            "Seasonally Adjusted"
            if "seasonally adjusted" in series_name
            else "Original"
        )
        name = series_name.split(";")[1].strip()
        long_name = 20
        separator = " " if len(name) < long_name else "\n"
        name = f"{separator}{name}"
        lfooter = f"Australia. {series_type.capitalize()} series. "

        quarterly = annualise_rates(
            abs_dict[table][series_id].pct_change(3), 4
        ).dropna()
        print(f"Quarterly annualised endpoint: {quarterly.iloc[-1]:0.2f}%")
        line_and_target_plot(
            quarterly,
            title=f"Quarterly Price Growth Annualised:{name}",
            ylabel="Per cent",
            lfooter=lfooter,
            rfooter=source,
            show=SHOW,
        )

        quarterly_s = hma(quarterly, henderson)
        print(f"Quarterly annualised & smoothed endpoint: {quarterly_s.iloc[-1]:0.2f}%")
        line_and_target_plot(
            quarterly_s,
            title=f"Quarterly Price Growth Annualised/Smoothed:{name}",
            ylabel="Per cent",
            lfooter=f"{lfooter}{henderson}-term Henderson moving average. ",
            rfooter=source,
            show=SHOW,
        )

        monthly = annualise_rates(abs_dict[table][series_id].pct_change(1), 12).dropna()
        print(f"Monthly annualised endpoint: {monthly.iloc[-1]:0.2f}%")
        line_and_target_plot(
            monthly,
            title=f"Monthly Price Growth Annualised:{name}",
            ylabel="Per cent",
            lfooter=lfooter,
            rfooter=source,
            show=SHOW,
        )

        monthly_s = hma(monthly, henderson)
        print(f"Monthly annualised & smoothed endpoint: {monthly_s.iloc[-1]:0.2f}%")
        line_and_target_plot(
            monthly_s,
            title=f"Monthly Price Growth Annualised/Smoothed:{name}",
            ylabel="Per cent",
            lfooter=f"{lfooter}{henderson}-term Henderson moving average. ",
            rfooter=source,
            show=SHOW,
        )
        print()

In [8]:
annualised_growth()

Index Numbers ;  All groups CPI ;  Australia ;
Quarterly annualised endpoint: 2.34%
Quarterly annualised & smoothed endpoint: 3.86%
Monthly annualised endpoint: 4.04%
Monthly annualised & smoothed endpoint: 2.95%

Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;
Quarterly annualised endpoint: 3.01%
Quarterly annualised & smoothed endpoint: 4.78%
Monthly annualised endpoint: 5.06%
Monthly annualised & smoothed endpoint: 4.05%



### Selected sub items

In [9]:
def food_fuel() -> None:
    """Produce food and fuel sub-item charts."""

    annual = "Corresponding Month of Previous Year"
    selectors = {
        "Food and non-alcoholic beverages": {
            "Food and non-alcoholic beverages": did_col,
            "Original": type_col,
            annual: did_col,
        },
        "Automotive fuel": {
            "Automotive fuel": did_col,
            "Original": type_col,
            annual: did_col,
        },
    }

    data = pd.DataFrame()
    for name, selector in selectors.items():
        series_id, _ = find_id(meta, selector, verbose=False)
        table = meta.loc[meta["Series ID"] == series_id, "Table"].values[0]
        data[name] = abs_dict[table][series_id]

    title = "Monthly CPI Indicator - Annual Growth - Food & Fuel"
    tag = "1"
    series_type = "Original series"
    line_plot(
        data,
        title=title,
        ylabel="Per cent",
        color=[COLOR_BLUE, COLOR_AMBER],
        width=1.5,
        tags=tag,
        legend={"loc": "best", "fontsize": "x-small"},
        rfooter=f"{source}",
        lfooter=f"Australia: {series_type}",
        show=SHOW,
    )


food_fuel()

In [10]:
def housing() -> None:
    """Produce housing related charts."""

    annual = "Corresponding Month of Previous Year"
    selectors = {
        "Rents": {
            "Rents": did_col,
            "Original": type_col,
            annual: did_col,
        },
        "New dwelling purchase by owner-occupiers": {
            "New dwelling purchase by owner-occupiers": did_col,
            "Original": type_col,
            annual: did_col,
        },
    }

    data = pd.DataFrame()
    for name, selector in selectors.items():
        series_id, _ = find_id(meta, selector, verbose=False)
        table = meta.loc[meta["Series ID"] == series_id, "Table"].values[0]
        data[name] = abs_dict[table][series_id]

    title = "Monthly CPI Indicator - Annual Growth - Residential"
    tag = "2"
    series_type = "Original series"
    line_plot(
        data,
        title=title,
        ylabel="Per cent",
        color=[COLOR_BLUE, COLOR_AMBER],
        width=1.5,
        tags=tag,
        legend={"loc": "best", "fontsize": "x-small"},
        rfooter=f"{source}",
        lfooter=f"Australia: {series_type}",
        show=SHOW,
    )


housing()

## Finished

In [11]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Jan 10 2024 12:09:41

Python implementation: CPython
Python version       : 3.12.1
IPython version      : 8.20.0

pandas    : 2.1.4
matplotlib: 3.8.2

Watermark: 2.4.3



In [12]:
print("Done")

Done
